```{contents}
```
## **Worker Agent in LangGraph**

A **Worker Agent** in LangGraph is a specialized autonomous component responsible for executing a well-defined task within a multi-agent system, under the coordination of a **Supervisor Agent**. Worker agents perform focused work such as research, computation, validation, tool execution, or content generation, and report results back through shared state.

---

### **1. Conceptual Role**

| Role             | Responsibility                       |
| ---------------- | ------------------------------------ |
| Supervisor Agent | Plans, delegates, resolves conflicts |
| **Worker Agent** | Executes assigned subtask            |
| Shared State     | Communication medium                 |

Worker agents are **not independent chatbots** — they are **nodes within a controlled execution graph**.

---

### **2. Why Worker Agents Matter**

Large tasks require decomposition:

```
Goal → Plan → [ Worker A | Worker B | Worker C ] → Merge → Final Output
```

Benefits:

* Parallelism
* Specialization
* Fault isolation
* Scalable design

---

### **3. Core Characteristics**

| Property          | Description                      |
| ----------------- | -------------------------------- |
| Task-specific     | Narrow responsibility            |
| Stateless logic   | Reads/writes shared state        |
| Tool-aware        | Can call tools                   |
| Isolated failures | One worker does not crash system |
| Composable        | Can be reused in multiple graphs |

---

### **4. Worker Agent Architecture**

```
            ┌──────────────┐
User → Supervisor Agent →  Worker Agents → Aggregator → Output
            └──────────────┘
```

Communication occurs **only via shared state**, not direct messaging.

---

### **5. Worker Agent Implementation**

#### **State Definition**

```python
class State(TypedDict):
    query: str
    research: str
    math_result: float
    final_answer: str
```

---

#### **Worker Functions**

```python
def research_worker(state: State):
    return {"research": f"Research on {state['query']}"}

def math_worker(state: State):
    return {"math_result": len(state["query"]) * 1.5}
```

---

#### **Supervisor Router**

```python
def supervisor(state: State):
    if "research" not in state:
        return "research_worker"
    if "math_result" not in state:
        return "math_worker"
    return "final"
```

---

#### **Graph Construction**

```python
builder = StateGraph(State)

builder.add_node("supervisor", supervisor)
builder.add_node("research_worker", research_worker)
builder.add_node("math_worker", math_worker)
builder.add_node("final", lambda s: {"final_answer": "done"})

builder.set_entry_point("supervisor")

builder.add_conditional_edges("supervisor", supervisor, {
    "research_worker": "research_worker",
    "math_worker": "math_worker",
    "final": "final"
})

builder.add_edge("research_worker", "supervisor")
builder.add_edge("math_worker", "supervisor")
builder.add_edge("final", END)
```

---

### **6. Execution Flow**

| Step | Action                     |
| ---- | -------------------------- |
| 1    | Supervisor evaluates state |
| 2    | Assigns next worker        |
| 3    | Worker performs task       |
| 4    | State updated              |
| 5    | Supervisor re-evaluates    |
| 6    | Loop until complete        |

---

### **7. Variants of Worker Agents**

| Variant            | Use Case              |
| ------------------ | --------------------- |
| Research Worker    | Retrieval & synthesis |
| Tool Worker        | API execution         |
| Validator Worker   | Fact checking         |
| Computation Worker | Math, code            |
| Generator Worker   | Text, reports         |
| Critic Worker      | Quality control       |
| Repair Worker      | Error correction      |

---

### **8. Production Design Guidelines**

| Concern       | Strategy                |
| ------------- | ----------------------- |
| Scalability   | Parallel workers        |
| Reliability   | Retry + fallback        |
| Safety        | Tool permission gating  |
| Observability | Trace each worker       |
| Cost          | Token budget per worker |

---

### **9. Why LangGraph Worker Agents Are Powerful**

Traditional agent frameworks allow agents to talk freely.
LangGraph enforces **deterministic orchestration**:

> **Supervisor decides → Worker executes → State updates → Repeat**

This provides:

* Predictability
* Debuggability
* Safety
* Production reliability

---

### **10. Mental Model**

A worker agent is a **microservice inside an AI system**:

> **Input → Function → State Update**


### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# -----------------------------
# 1. Shared State
# -----------------------------
class State(TypedDict, total=False):
    query: str
    research: str
    math_result: float
    final_answer: str

# -----------------------------
# 2. Worker Agents
# -----------------------------
def research_worker(state: State):
    return {"research": f"Research completed for: {state['query']}"}

def math_worker(state: State):
    return {"math_result": len(state["query"]) * 1.5}

# -----------------------------
# 3. Final Aggregator
# -----------------------------
def final_node(state: State):
    answer = f"{state['research']} | Score: {state['math_result']}"
    return {"final_answer": answer}

# -----------------------------
# 4. Router (NO state updates)
# -----------------------------
def router(state: State):
    if "research" not in state:
        return "research_worker"
    if "math_result" not in state:
        return "math_worker"
    return "final"

# -----------------------------
# 5. Build Graph
# -----------------------------
builder = StateGraph(State)

builder.add_node("research_worker", research_worker)
builder.add_node("math_worker", math_worker)
builder.add_node("final", final_node)

builder.set_entry_point("research_worker")

builder.add_edge("research_worker", "math_worker")
builder.add_edge("math_worker", "final")
builder.add_edge("final", END)

graph = builder.compile()

# -----------------------------
# 6. Run
# -----------------------------
result = graph.invoke({"query": "LangGraph worker agents"})
print(result["final_answer"])


Research completed for: LangGraph worker agents | Score: 34.5
